## Supplementary Notebook - Linear Modeling using patsy

This notebook demonstrate how to create a design matrix using the [patsy](https://patsy.readthedocs.io/en/latest/) library. This library is very useful for specifying exactly the predictors and interactions that you want for you model.

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from patsy import dmatrix

In [ ]:
penguins = pd.read_csv('data/penguins.csv').dropna().reset_index(drop = True)

In [ ]:
penguins[['species', 'bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'sex', 'body_mass_g']].head()

We'll start with a baseline model to predict body mass based on flipper length alone.

In [ ]:
variables = ['flipper_length_mm']

X = penguins[variables]
y = penguins['body_mass_g']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 321, stratify = penguins['species'])

linreg = LinearRegression().fit(X_train, y_train)

In [ ]:
coefficients = pd.DataFrame({
    'variable': ['intercept'] + list(X_train.columns),
    'coefficient': [linreg.intercept_] + list(linreg.coef_)
})
coefficients

In [ ]:
x_grid = np.linspace(start = penguins['flipper_length_mm'].min(),
                    stop = penguins['flipper_length_mm'].max(),
                    num = 150)
y_grid = linreg.predict(x_grid.reshape(-1,1))


fig, ax = plt.subplots(figsize = (10,6))
plt.plot(x_grid, y_grid)
sns.scatterplot(data = penguins,
             x = 'flipper_length_mm',
             y = 'body_mass_g', ax = ax);
#plt.tight_layout()
#plt.savefig('images/scatter_01.png', dpi = 150);

In [ ]:
x_grid = np.linspace(start = penguins['flipper_length_mm'].min(),
                    stop = penguins['flipper_length_mm'].max(),
                    num = 150)
y_grid = linreg.predict(x_grid.reshape(-1,1))


fig, ax = plt.subplots(figsize = (10,6))
plt.plot(x_grid, y_grid)
sns.scatterplot(data = penguins,
             x = 'flipper_length_mm',
             y = 'body_mass_g', 
                hue = 'species',
                palette = ['red', 'blue', 'limegreen'],
                ax = ax);
#plt.tight_layout()
#plt.savefig('images/scatter_02.png', dpi = 150);

In [ ]:
print(f'MSE: {mean_squared_error(y_test, linreg.predict(X_test))}')
print(f'R2: {r2_score(y_test, linreg.predict(X_test))}')

Now, we'll add on the species information. We can create our predictors matrix using the `dmatrix` function and specifying a [formula](https://patsy.readthedocs.io/en/latest/formulas.html). Here, we want a column for flipper length and dummy columns for species.

Note that dmatrix will add an intercept column by default, so we need to tell the LinearRegression model to not fit the intercept.

In [ ]:
X = dmatrix('~flipper_length_mm + species', data = penguins, return_type = 'dataframe')
y = penguins['body_mass_g']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 321, stratify = penguins['species'])

linreg = LinearRegression(fit_intercept = False).fit(X_train, y_train)

In [ ]:
coefficients = pd.DataFrame({
    'variable': list(X_train.columns),
    'coefficient': list(linreg.coef_)
})
coefficients

In [ ]:
print(f'MSE: {mean_squared_error(y_test, linreg.predict(X_test))}')
print(f'R2: {r2_score(y_test, linreg.predict(X_test))}')

In [ ]:
fig, ax = plt.subplots(figsize = (10,6))

for species, color in zip(penguins['species'].unique(), ['red', 'blue', 'limegreen']):
    
    intercept = coefficients.set_index('variable').loc['Intercept'].values[0]
    
    if species != 'Adelie':
        intercept += coefficients.set_index('variable').loc[f'species[T.{species}]'].values[0]

    x_grid = np.linspace(start = penguins[penguins['species'] == species]['flipper_length_mm'].min(),
                        stop = penguins[penguins['species'] == species]['flipper_length_mm'].max(),
                        num = 150)
    y_grid = intercept + coefficients.set_index('variable').loc['flipper_length_mm'].values[0]*x_grid

    plt.plot(x_grid, y_grid, #label = species, 
             color = color)
    
    
sns.scatterplot(data = penguins,
             x = 'flipper_length_mm',
             y = 'body_mass_g', 
                hue = 'species',
                palette = ['red', 'blue', 'limegreen'],
                ax = ax);
#plt.tight_layout()
#plt.savefig('images/scatter_03.png', dpi = 150);

We can add an interaction between flipper length and species by adding a term with a colon.

In [ ]:
X = dmatrix('flipper_length_mm + species + flipper_length_mm:species', 
        data = penguins, 
        return_type = 'dataframe')
y = penguins['body_mass_g']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 321, stratify = penguins['species'])

linreg = LinearRegression(fit_intercept = False).fit(X_train, y_train)

In [ ]:
coefficients = pd.DataFrame({
    'variable': list(X_train.columns),
    'coefficient': list(linreg.coef_)
})
coefficients

In [ ]:
print(f'MSE: {mean_squared_error(y_test, linreg.predict(X_test))}')
print(f'R2: {r2_score(y_test, linreg.predict(X_test))}')

In [ ]:
fig, ax = plt.subplots(figsize = (10,6))

for species, color in zip(penguins['species'].unique(), ['red', 'blue', 'limegreen']):
    
    intercept = coefficients.set_index('variable').loc['Intercept'].values[0]
    slope = coefficients.set_index('variable').loc['flipper_length_mm'].values[0]
    
    if species != 'Adelie':
        intercept += coefficients.set_index('variable').loc[f'species[T.{species}]'].values[0]
        slope += coefficients.set_index('variable').loc[f'flipper_length_mm:species[T.{species}]'].values[0]

    x_grid = np.linspace(start = penguins[penguins['species'] == species]['flipper_length_mm'].min(),
                        stop = penguins[penguins['species'] == species]['flipper_length_mm'].max(),
                        num = 150)
    y_grid = intercept + slope*x_grid

    plt.plot(x_grid, y_grid, #label = species, 
             color = color)
    
    
sns.scatterplot(data = penguins,
             x = 'flipper_length_mm',
             y = 'body_mass_g', 
                hue = 'species',
                palette = ['red', 'blue', 'limegreen'],
                ax = ax);
#plt.tight_layout()
#plt.savefig('images/scatter_04.png', dpi = 150);

Now, let's add the sex variable in.

In [ ]:
X = dmatrix('flipper_length_mm + species + sex + flipper_length_mm:species + flipper_length_mm:sex', 
        data = penguins, 
        return_type = 'dataframe')
y = penguins['body_mass_g']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 321, stratify = penguins['species'])

linreg = LinearRegression(fit_intercept = False).fit(X_train, y_train)

In [ ]:
coefficients = pd.DataFrame({
    'variable': list(X_train.columns),
    'coefficient': list(linreg.coef_)
})
coefficients

In [ ]:
print(f'MSE: {mean_squared_error(y_test, linreg.predict(X_test))}')
print(f'R2: {r2_score(y_test, linreg.predict(X_test))}')

In [ ]:
fig, axes = plt.subplots(figsize = (10,6), nrows= 2)

for sex, ax in zip(['female', 'male'], axes):
    
    ax.set_title(sex)
    
    for species, color in zip(penguins['species'].unique(), ['red', 'blue', 'limegreen']):
        
        intercept = coefficients.set_index('variable').loc['Intercept'].values[0]
        slope = coefficients.set_index('variable').loc['flipper_length_mm'].values[0]

        if species != 'Adelie':
            intercept += coefficients.set_index('variable').loc[f'species[T.{species}]'].values[0]
            slope += coefficients.set_index('variable').loc[f'flipper_length_mm:species[T.{species}]'].values[0]

        if sex == 'male':
            intercept += coefficients.set_index('variable').loc['sex[T.male]'].values[0]
            slope += coefficients.set_index('variable').loc['flipper_length_mm:sex[T.male]'].values[0]


        x_grid = np.linspace(start = penguins[(penguins['species'] == species) & (penguins['sex'] == sex)]['flipper_length_mm'].min(),
                            stop = penguins[(penguins['species'] == species) & (penguins['sex'] == sex)]['flipper_length_mm'].max(),
                            num = 150)
        
        y_grid = intercept + slope*x_grid
                
        ax.plot(x_grid, y_grid, #label = species, 
                 color = color)
    
    sns.scatterplot(data = penguins[penguins['sex'] == sex],
                 x = 'flipper_length_mm',
                 y = 'body_mass_g', 
                    hue = 'species',
                    palette = ['red', 'blue', 'limegreen'],
                    ax = ax);
    
    
#plt.tight_layout()
#plt.savefig('images/scatter_05.png', dpi = 150);